In [1]:
from matplotlib import pyplot as plt
from torch.utils import data
import numpy as np
import pandas as pd

#Organize Data
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
#Find Hyperparameters
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from ast import literal_eval

Mounted at /content/gdrive


In [4]:
#Connect to data folder of NerLTR-DTA
root_path = '/data/'
import os
os.chdir(root_path)
!pwd

/content/gdrive/My Drive/KI JKU/Seminar_PracticalW_MasterThesis/MasterThesis/NerLTR/data


In [5]:
# install lifelines
!pip install -q lifelines
from lifelines.utils import concordance_index
from sklearn.metrics import mean_squared_error, r2_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [6]:
!pip3 install pickle5
import pickle5 as pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=262127 sha256=8016582548a1872223a7f1c34003663e83d24ab5c1f0d4684c34f283fe8d9645
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


#Paths

In [7]:
#paths
#featues
path_train_fold1 = 'features/combined/train_fold1_KIBA.txt'
path_train_fold2 = 'features/combined/train_fold2_KIBA.txt'
path_train_fold3 = 'features/combined/train_fold3_KIBA.txt'
path_train_fold4 = 'features/combined/train_fold4_KIBA.txt'
path_train_fold5 = 'features/combined/train_fold5_KIBA.txt'
path_test = 'features/test_features_KIBA.txt'

#Affinities
path_train_fold1_aff_KIBA = 'affi/train_aff_fold1_KIBA.pkl'
path_train_fold2_aff_KIBA = 'affi/train_aff_fold2_KIBA.pkl'
path_train_fold3_aff_KIBA = 'affi/train_aff_fold3_KIBA.pkl'
path_train_fold4_aff_KIBA = 'affi/train_aff_fold4_KIBA.pkl'
path_train_fold5_aff_KIBA = 'affi/train_aff_fold5_KIBA.pkl'
path_test_aff_KIBA = 'affi/test_aff_KIBA.pkl'

#Get Labels

In [8]:
#KIBA  fold 1-4 = train; fold 5 = val 
#folds
train_affi_fold1_KIBA = pd.read_pickle(path_train_fold1_aff_KIBA)
y_train_fold1_KIBA = train_affi_fold1_KIBA['affinity'].tolist()
train_affi_fold2_KIBA = pd.read_pickle(path_train_fold2_aff_KIBA)
y_train_fold2_KIBA = train_affi_fold2_KIBA['affinity'].tolist()
train_affi_fold3_KIBA = pd.read_pickle(path_train_fold3_aff_KIBA)
y_train_fold3_KIBA = train_affi_fold3_KIBA['affinity'].tolist()
train_affi_fold4_KIBA = pd.read_pickle(path_train_fold4_aff_KIBA)
y_train_fold4_KIBA = train_affi_fold4_KIBA['affinity'].tolist()
train_affi_fold5_KIBA = pd.read_pickle(path_train_fold5_aff_KIBA)
y_train_fold5_KIBA = train_affi_fold5_KIBA['affinity'].tolist()

y_train_KIBA = y_train_fold1_KIBA#+y_train_fold2_KIBA+y_train_fold3_KIBA+y_train_fold4_KIBA+y_train_fold5_KIBA

#test
test_affi_KIBA = pd.read_pickle(path_test_aff_KIBA)
y_test_KIBA = test_affi_KIBA['affinity'].tolist()

#lengths
print(f'Len Fold1:{len(y_train_fold1_KIBA)}')
print(f'Len Fold2:{len(y_train_fold2_KIBA)}')
print(f'Len Fold3:{len(y_train_fold3_KIBA)}')
print(f'Len Fold4:{len(y_train_fold4_KIBA)}')
print(f'Len Fold5:{len(y_train_fold5_KIBA)}')

Len Fold1:19709
Len Fold2:19709
Len Fold3:19709
Len Fold4:19709
Len Fold5:19709


#Get IDs

In [9]:
#KIBA  fold 1-4 = train; fold 5 = val 
#folds
y_train_fold1_KIBA_ID = train_affi_fold1_KIBA['target_ID'].tolist()
y_train_fold2_KIBA_ID = train_affi_fold2_KIBA['target_ID'].tolist()
y_train_fold3_KIBA_ID = train_affi_fold3_KIBA['target_ID'].tolist()
y_train_fold4_KIBA_ID = train_affi_fold4_KIBA['target_ID'].tolist()
y_train_fold5_KIBA_ID = train_affi_fold5_KIBA['target_ID'].tolist()

y_train_KIBA_ID = y_train_fold1_KIBA_ID#+y_train_fold2_KIBA_ID+y_train_fold3_KIBA_ID+y_train_fold4_KIBA_ID+y_train_fold5_KIBA_ID

#test
y_test_KIBA_ID = test_affi_KIBA['target_ID'].tolist()

#lengths
print(f'Len Fold1:{len(y_train_fold1_KIBA_ID)}')
print(f'Len Fold2:{len(y_train_fold2_KIBA_ID)}')
print(f'Len Fold3:{len(y_train_fold3_KIBA_ID)}')
print(f'Len Fold4:{len(y_train_fold4_KIBA_ID)}')
print(f'Len Fold5:{len(y_train_fold5_KIBA_ID)}')
print(f'Len Test:{len(y_test_KIBA_ID)}')

Len Fold1:19709
Len Fold2:19709
Len Fold3:19709
Len Fold4:19709
Len Fold5:19709
Len Test:19709


#Get Features

In [10]:
#folds drugs
train_fold1=np.loadtxt(path_train_fold1)
train_fold2=np.loadtxt(path_train_fold2)
train_fold3=np.loadtxt(path_train_fold3)
train_fold4=np.loadtxt(path_train_fold4)
train_fold5=np.loadtxt(path_train_fold5)
test_data=np.loadtxt(path_test)

In [11]:
print(f'Len Fold1: {len(train_fold1[0])}')
print(f'Len Test: {len(test_data)}')

Len Fold1: 908
Len Test: 19709


In [12]:
args1=(train_fold1,train_fold2,train_fold3,train_fold4,train_fold5)
train_data=train_fold1#np.concatenate(args1)

In [13]:
print(f'Len Trainset: {len(train_data)}')

Len Trainset: 19709


#Hyperpara Search

In [ ]:
GBR=GradientBoostingRegressor()
n_estimators = [200, 400, 600 ,800]
max_depth = [5, 10, 15 , 20]
min_samples_leaf = [1, 4, 8] 
min_samples_split = [6, 10, 14]
max_features = ['sqrt']
learning_rate = [0.1, 0.05, 0.01]

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features,
    'learning_rate':learning_rate}


In [ ]:
random_search =RandomizedSearchCV(GradientBoostingRegressor(loss='squared_error'), param_grid, random_state=1, n_iter=50, cv=5, verbose=3, n_jobs=1)
random_search.fit(train_data, y_train_KIBA)
random_search.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.499 total time=  22.8s
[CV 2/5] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.491 total time=  21.9s
[CV 3/5] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.472 total time=  21.9s
[CV 4/5] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.478 total time=  21.7s
[CV 5/5] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.481 total time=  21.7s
[CV 1/5] END learning_rate=0.01, max_depth=15, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=400;, score=0.602 total time= 1.

{'n_estimators': 600,
 'min_samples_split': 14,
 'min_samples_leaf': 8,
 'max_features': 'sqrt',
 'max_depth': 10,
 'learning_rate': 0.05}

#Train on Full train data

In [14]:
#features
args2=(train_fold1,train_fold2,train_fold3,train_fold4,train_fold5)
train_full=np.concatenate(args2)

#Labels
y_train_full = y_train_fold1_KIBA+y_train_fold2_KIBA+y_train_fold3_KIBA+y_train_fold4_KIBA+y_train_fold5_KIBA

In [15]:
seeds = np.load('seeds.npy')
pred_KIBA=[]
for seed in seeds:
  #Fit
  best_params = {'n_estimators': 800,
                'min_samples_split': 10,
                'min_samples_leaf': 8,
                'max_features': 'sqrt',
                'max_depth': 15,
                'learning_rate': 0.01,
                'random_state': seed}
  classifier_KIBA = GradientBoostingRegressor(**best_params)
  classifier_KIBA.fit(train_full, y_train_full)
  pred_KIBA.append(classifier_KIBA.predict(test_data))

In [16]:
pred_KIBA

[array([13.2545531 , 12.65724542, 11.46585802, ..., 11.4480296 ,
        11.53263049, 11.52376433]),
 array([13.15033377, 12.7068249 , 11.44232757, ..., 11.42198041,
        11.48598063, 11.49599053]),
 array([13.1611207 , 12.68797179, 11.42200694, ..., 11.41008168,
        11.52087392, 11.49523401]),
 array([13.29990124, 12.63467273, 11.4687768 , ..., 11.42052219,
        11.47202629, 11.5054276 ]),
 array([13.18281558, 12.67942245, 11.48091825, ..., 11.40396456,
        11.49336259, 11.51727148]),
 array([13.23292774, 12.75350657, 11.44482158, ..., 11.44184427,
        11.54259552, 11.46338519]),
 array([13.15758527, 12.67052443, 11.45719524, ..., 11.40946782,
        11.47832937, 11.48400697]),
 array([13.14803188, 12.6842954 , 11.41000339, ..., 11.40309183,
        11.51561109, 11.48138754]),
 array([13.18220261, 12.67479751, 11.39666475, ..., 11.40923983,
        11.48940712, 11.49099018]),
 array([13.19421293, 12.61838057, 11.42478955, ..., 11.42854233,
        11.48424495, 11.50

#Metrics

In [17]:
def r_squared_error(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    y_pred_mean = [np.mean(y_pred) for y in y_pred]

    mult = sum((y_pred - y_pred_mean) * (y_obs - y_obs_mean))
    mult = mult * mult

    y_obs_sq = sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))
    y_pred_sq = sum((y_pred - y_pred_mean) * (y_pred - y_pred_mean) )

    return mult / float(y_obs_sq * y_pred_sq)

def squared_error_zero(y_obs,y_pred):
    k = get_k(y_obs,y_pred)

    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)
    y_obs_mean = [np.mean(y_obs) for y in y_obs]
    upp = sum((y_obs - (k*y_pred)) * (y_obs - (k* y_pred)))
    down= sum((y_obs - y_obs_mean)*(y_obs - y_obs_mean))

    return 1 - (upp / float(down))

def get_k(y_obs,y_pred):
    y_obs = np.array(y_obs)
    y_pred = np.array(y_pred)

    return sum(y_obs*y_pred) / float(sum(y_pred*y_pred))

def get_rm2(ys_orig,ys_line):
    r2 = r_squared_error(ys_orig, ys_line)
    r02 = squared_error_zero(ys_orig, ys_line)
    return r2 * (1 - np.sqrt(np.absolute((r2*r2)-(r02*r02))))
    
import torch 
def concordance_index_taskwise(predictions, labels, target_ids):
    CIs = list()
    target_id_list = list()
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)
    target_ids = torch.tensor(target_ids)

    for target_idx in torch.unique(target_ids):
        rows = torch.where(target_ids == target_idx)
        preds = predictions[rows]
        y = labels[rows]

        if torch.unique(y).shape[0] >= 2:
            ci = concordance_index(y, preds)
            CIs.append(ci)
            target_id_list.append(target_idx.item())
        else:
            CIs.append(np.nan)
            target_id_list.append(target_idx.item())
    return np.nanmean(CIs), CIs, target_id_list

In [18]:
cis=[]
mses=[]
for i, seed in enumerate(seeds):
  print(f"Seed: {seed}")
  ci, CIs, _ = concordance_index_taskwise(pred_KIBA[i], y_test_KIBA, y_test_KIBA_ID)
  print(f'Concordance index:\t {ci}')
  cis.append(ci)
  mse=mean_squared_error(y_test_KIBA, pred_KIBA[i])
  print(f"MSE\t\t：{mse}")
  mses.append(mse)
  print(f"rm2\t\t：{get_rm2(y_test_KIBA, pred_KIBA[i])}")

Seed: 658
Concordance index:	 0.855702196894727
MSE		：0.17535033753873977
rm2		：0.6999629003349092
Seed: 25
Concordance index:	 0.8569121932811433
MSE		：0.1760323382590237
rm2		：0.6994710509788628
Seed: 502
Concordance index:	 0.8587763197108501
MSE		：0.17530384357990964
rm2		：0.7007044739328422
Seed: 861
Concordance index:	 0.8550040605138737
MSE		：0.17450859437056385
rm2		：0.7007989390486459
Seed: 696
Concordance index:	 0.8561843951495596
MSE		：0.1749403399926676
rm2		：0.7009062854106305
Seed: 729
Concordance index:	 0.8553255796865478
MSE		：0.17563960832368058
rm2		：0.6994383799903441
Seed: 647
Concordance index:	 0.8547414222814375
MSE		：0.1754730204984531
rm2		：0.7002717579865385
Seed: 818
Concordance index:	 0.8540602781052331
MSE		：0.17548786468824645
rm2		：0.7003015904726869
Seed: 83
Concordance index:	 0.8560705673304794
MSE		：0.1747473269099684
rm2		：0.7008924879494467
Seed: 153
Concordance index:	 0.8568099822486304
MSE		：0.17553553371323044
rm2		：0.6998156433798469


In [22]:
np.save("GBT_ci.npy", cis)
np.save("GBT_mse.npy", mses)

In [20]:
concordance_index(y_test_KIBA, pred_KIBA)

ValueError: ignored

In [ ]:
len(CIs)
CIs